# Page Visits Funnel
Cool T-Shirts Inc. has asked you to analyze data on visits to their website. Your job is to build a funnel, which is a description of how many people continue to the next step of a multi-step process.

In this case, our funnel is going to describe the following process:

A user visits CoolTShirts.com
A user adds a t-shirt to their cart
A user clicks “checkout”
A user actually purchases a t-shirt

In [6]:
import pandas as pd

Import all the files

In [7]:
visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
                   
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

In [8]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Step 2: Left merging visits and cart

In [9]:
visit_carts = pd.merge(
  visits,
  cart,
  how = 'left'
)
print(visit_carts)

                                   user_id          visit_time  \
0     943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1     0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2     6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3     6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4     a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   
...                                    ...                 ...   
1995  33913ac2-03da-45ae-8fc3-fea39df827c6 2017-03-25 03:29:00   
1996  4f850132-b99d-4623-80e6-6e61d003577e 2017-01-08 09:57:00   
1997  f0830b9b-1f5c-4e74-b63d-3f847cc6ce70 2017-09-07 12:56:00   
1998  b01bffa7-63ba-4cd3-9d93-eb1477c23831 2017-07-20 04:37:00   
1999  0336ca81-8d68-443f-9248-ac0b8ad147d5 2017-11-15 10:11:00   

               cart_time  
0                    NaT  
1    2017-01-26 14:44:00  
2    2017-08-20 08:31:00  
3                    NaT  
4                    NaT  
...                  ...  
1995              

Step 3: How long is `visits_cart`?

In [10]:
print(len(visit_carts))

2000


Step 4: How many timestamps are null for `cart_time`?

In [11]:
null_timestamp = visit_carts[visit_carts['cart_time'].isnull()]
print(len(null_timestamp))

1652


Step 5: What percentage only visited?

In [13]:
cart_time_null = visit_carts[visit_carts['cart_time'].isnull()]

all_null = cart_time_null['user_id'].count()
all_visit = visit_carts['user_id'].count()

percentage_null = (all_null/all_visit)*100
print('The percentage that of users who visited the website but not placing a t-shirt in their cart is {}%\n'.format(percentage_null))


The percentage that of users who visited the website but not placing a t-shirt in their cart is 82.6%



Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [14]:
cart_checkout = pd.merge(
  cart,
  checkout,
  how='left'
)
#print(cart_checkout)
checkout_time_null = cart_checkout[cart_checkout['checkout_time'].isnull()]

all_checkout_null = len(checkout_time_null)
all_cart = len(cart_checkout)

percentage_checkout = round(all_checkout_null*100/all_cart,2)

print('The percentage of users put items in their cart, but did not proceed to checkout is {}%\n'.format(percentage_checkout))


The percentage of users put items in their cart, but did not proceed to checkout is 35.06%



Step 7: Merge it all together

In [16]:
all_data = visit_carts.merge(
  cart_checkout,
  how='left'
).merge(
  purchase,
  how='left'
)


Step 8: % of users who got to checkout but did not purchase

In [17]:
all_data_checkout = all_data.loc[pd.notna(all_data['checkout_time'])]

all_data_checkout_purchase = all_data_checkout[all_data_checkout['purchase_time'].isnull()]

len_all_data = len(all_data)
len_all_data_checkout_purchase = len(all_data_checkout_purchase)

percentage_all_data_null = round(len_all_data_checkout_purchase * 100/ len_all_data,2)

print('The percentage of users proceeded to checkout, but did not purchase a t-shirt is {}%\n'.format(percentage_all_data_null))

The percentage of users proceeded to checkout, but did not purchase a t-shirt is 3.89%



Step 9: check each part of the funnel, let's print all 3 of them again

In [18]:
print('The highest percentage is visiting the website but not placing a t-shirt in their cart.\n')

The highest percentage is visiting the website but not placing a t-shirt in their cart.



*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: adding new column

In [19]:
all_data['average_time'] = all_data['purchase_time']-all_data['visit_time']

Step 11: examine the results

In [20]:
print(all_data)

                                   user_id          visit_time  \
0     943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1     0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2     6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3     6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4     a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   
...                                    ...                 ...   
2103  33913ac2-03da-45ae-8fc3-fea39df827c6 2017-03-25 03:29:00   
2104  4f850132-b99d-4623-80e6-6e61d003577e 2017-01-08 09:57:00   
2105  f0830b9b-1f5c-4e74-b63d-3f847cc6ce70 2017-09-07 12:56:00   
2106  b01bffa7-63ba-4cd3-9d93-eb1477c23831 2017-07-20 04:37:00   
2107  0336ca81-8d68-443f-9248-ac0b8ad147d5 2017-11-15 10:11:00   

               cart_time       checkout_time       purchase_time  \
0                    NaT                 NaT                 NaT   
1    2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00   
2  

Step 12: average time to purchase

In [22]:
avg_purchase_time = all_data['purchase_time'].mean()
print('The average time to purchase is {}'.format(avg_purchase_time))

The average time to purchase is 2017-06-12 07:04:02.380952320
